In [1]:
!pip install googletrans==3.1.0a0
!pip install langdetect
!pip install ai4bharat.transliteration
from googletrans import Translator
from langdetect import detect
from ai4bharat.transliteration import XlitEngine
import os
#from humanize import number
from datetime import datetime as dt
import pandas as pd, numpy as np, os.path as op
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("\n*****Required libraries imported*****")

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16414 sha256=13731e06c608a5189cf914cad55f506717945e9ea8db8c61758219c128f0e15d
  Stored in directory: c:\users\akash\appdata\local\pip\cache\wheels\81\f2\e0\d578821d723b473d18610ea93810e4a5402463919f07e603d9
Successfully built googletrans


DEPRECATION: omegaconf 2.0.5 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: omegaconf 2.0.5 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.5 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Defaulting to user installation because normal site-packages is not writeable

*****Required libraries imported*****


In [2]:
data1 = pd.read_excel(r'C:\Users\akash\Documents\Haqdarshak\Work\Nassscom\1.0\NF 1.0 Local Language Names.xlsx')
#data2 = pd.read_csv(r'C:\Users\akash\Documents\Haqdarshak\Work\Nassscom\1.0\Pankhi_Nasscom1_unique_till_23_Apr.xlsx')

In [3]:
data1.head()

,Citizen GUID,Citizen Name
0,PN01XPX5,प्रकाश ठाकुर
1,PN01XT5B,झाडूराम साहू
2,PN01YFX0,ज्योति Devi
3,PN01ZDVO,सजीला कुमारी
4,PN01ZE65,पूनम Devi


In [5]:
def translit(col):
  #translator = Translator()
  
  x=col
  transliterated = []

  for s in x:
    if len(s.split())>1:
      ts = []
      for w in s.split():
        #translation = translator.detect(w)
        e = XlitEngine(src_script_type="indic", beam_width=10, rescore=False)
        ts.append(e.translit_word(w, topk=1))
      transliterated.append(' '.join(ts))
    else:
      e = XlitEngine(src_script_type="indic", beam_width=10, rescore=False)
      transliterated.append(e.translit_sentence(s, lang_code = translation.src))


  return transliterated


In [4]:
def translit(x):
  translator = Translator()
  lang = translator.translate(x)
  lang_code = lang.src
  e = XlitEngine(src_script_type="indic", beam_width = 5, rescore=False)

  try:
    return e.translit_sentence(x, lang_code = lang_code)
  except NotImplementedError:
    return x

In [5]:
data1['Translated'] = data1['Citizen GUID'].apply(lambda x: translit(x))

Initializing Multilingual model for transliteration


ValueError: mutable default <class 'fairseq.dataclass.configs.CommonConfig'> for field common is not allowed: use default_factory

In [ ]:
data1.head()

# Removing duplicates from citizen GUID

In [8]:
data2 = data2.drop(index = data2[data2.duplicated(['Citizen GUID'], keep='last')].index)

# English Alphabet list with is_en() function

In [27]:
import string 
alphabets = list(string.ascii_uppercase) + list(string.ascii_lowercase)
alphabets
def is_en(x):
    if x in alphabets:
        return True
    else:
        return False

# Getting non-eng citizens list

In [29]:
data2['Citizen Name'] = data2['Citizen Name'].fillna('a')
non_eng = data2[data2['Citizen Name'].apply(lambda x: False if is_en(x[0]) else True)]
non_eng.head()

,Citizen GUID,Citizen Name,Createdon,Gender,Mobile,Age,Family GUID,Family Name,HD ID,HD Name
110,PN01XPX5,प्रकाश ठाकुर,15-Feb-2023,M,6268638039,36.0,PG01QHI8,ठाकुर,gitanjalithakur,GITANJALI THAKUR
160,PN01XT5B,झाडूराम साहू,16-Feb-2023,M,8815158475,71.0,PG01QKE7,साहू,gitanjalithakur,GITANJALI THAKUR
451,PN01YFX0,ज्योति Devi,21-Feb-2023,F,8210834502,54.0,PG01R58A,Devi,soni.kumari.2,Soni Kumari
566,PN01ZDVO,सजीला कुमारी,27-Feb-2023,F,9708848735,8.0,PG01S1PO,Devi,soni.kumari.2,Soni Kumari
568,PN01ZE65,पूनम Devi,27-Feb-2023,F,8630773868,40.0,PG01S1Z3,Devi,soni.kumari.2,Soni Kumari


In [30]:
non_eng.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34677 entries, 110 to 1162031
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Citizen GUID  34677 non-null  object 
 1   Citizen Name  34677 non-null  object 
 2   Createdon     34677 non-null  object 
 3   Gender        34677 non-null  object 
 4   Mobile        27177 non-null  object 
 5   Age           34677 non-null  float64
 6   Family GUID   34677 non-null  object 
 7   Family Name   34677 non-null  object 
 8   HD ID         34677 non-null  object 
 9   HD Name       34677 non-null  object 
dtypes: float64(1), object(9)
memory usage: 2.9+ MB


In [33]:
non_eng.to_excel(r'C:\Users\akash\Downloads\NF 1.0 NonEng Citizen List.xlsx', index=False)

In [74]:
data2.rename(columns={'Mobile':'Screening Mobile'}, inplace=True)

In [75]:
data2.head()

,Citizen GUID,Citizen Name,Screening Mobile,Family GUID,Family Name,HD ID,HD Name,Opsco name
0,PN01WU0W,Sohga Bariha,8720036653,PG01POF9,Bariha,leambajagat,Leamba Jagat,Kshitij Kumar
1,PN01WU8K,Mayavati Bariha,8720036653,PG01POF9,Bariha,leambajagat,Leamba Jagat,Kshitij Kumar
2,PN01WUSE,Pushpa Diwan,9329853104,PG01PP51,Diwan,leambajagat,Leamba Jagat,Kshitij Kumar
3,PN01WVPG,Shrada Jagat,6265658782,PG01PPYC,Jagat,leambajagat,Leamba Jagat,Kshitij Kumar
4,PN01WW2T,Damrudhar Singh Jagat,8319814412,PG01PQAA,Jagat,leambajagat,Leamba Jagat,Kshitij Kumar


In [76]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425300 entries, 0 to 425299
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Case Id                425300 non-null  object        
 1   Createdon              425300 non-null  datetime64[ns]
 2   Scheme/Doc             425300 non-null  object        
 3   Scheme/Doc GUID        425300 non-null  object        
 4   Status                 425300 non-null  object        
 5   Docket Submitted Date  396531 non-null  datetime64[ns]
 6   Benefit received Date  388401 non-null  datetime64[ns]
 7   HD Suspected Cases     425300 non-null  object        
 8   State                  425300 non-null  object        
 9   District               425300 non-null  object        
 10  Citizen GUID           425300 non-null  object        
 11  Citizen Name           425300 non-null  object        
 12  Gender                 425300 non-null  obje

In [77]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173805 entries, 0 to 1173804
Data columns (total 8 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   Citizen GUID      1173805 non-null  object
 1   Citizen Name      1173800 non-null  object
 2   Screening Mobile  1154166 non-null  object
 3   Family GUID       1173804 non-null  object
 4   Family Name       1173798 non-null  object
 5   HD ID             1173441 non-null  object
 6   HD Name           1173804 non-null  object
 7   Opsco name        1173358 non-null  object
dtypes: object(8)
memory usage: 71.6+ MB


In [78]:
data2[['Citizen GUID', 'Screening Mobile']].head()

,Citizen GUID,Screening Mobile
0,PN01WU0W,8720036653
1,PN01WU8K,8720036653
2,PN01WUSE,9329853104
3,PN01WVPG,6265658782
4,PN01WW2T,8319814412


In [80]:
data3 = data2[['Citizen GUID', 'Screening Mobile']].drop_duplicates(subset=['Citizen GUID'])

In [81]:
data4 = pd.merge(data1, data3, on = 'Citizen GUID', how='left')

In [82]:
#data1.drop('Screening Mobile', axis=1, inplace=True)
data4.head()

,Case Id,Createdon,Scheme/Doc,Scheme/Doc GUID,Status,Docket Submitted Date,Benefit received Date,HD Suspected Cases,State,District,Citizen GUID,Citizen Name,Gender,Mobile,Screening Mobile_x,Age,Family GUID,Family Name,HD ID,HD Name,Opsco name,Scheme Category,duplicate,Screening Mobile_y
0,EP01SUJR,2023-02-28,Digital Health ID card_MP,SH0009I2,Benefit Received,2023-10-21,2023-10-21,Not Suspected,Madhya Pradesh,Damoh,PN01ZCK0,Parasmani Sahu,Male,9630483273,NaN,27,PG01S0GV,Sahu,parash.sahu,Parasmani Sahu,Parasmani Sahu,E-Gov,digital health id card_mpparasmani sahu9630483273,9630483273
1,EP01ZSED,2023-03-28,MP-Aadhar seeding to Samagra,DC00097P,Open,NaT,NaT,Not Suspected,Madhya Pradesh,Damoh,PN026FQH,Naresh Kumar Ahirwar,Male,9625601462,NaN,24,PG01YRH3,Ahirwar,parash.sahu,Parasmani Sahu,Parasmani Sahu,E-Gov,mp-aadhar seeding to samagranaresh kumar ahirw...,9625601462
2,EP0203IG,2023-03-29,E-Shram Card_MP,SH0009HI,Open,NaT,NaT,Not Suspected,Madhya Pradesh,Damoh,PN01ZCK0,Parasmani Sahu,Male,9630483273,NaN,27,PG01S0GV,Sahu,parash.sahu,Parasmani Sahu,Parasmani Sahu,E-Gov,e-shram card_mpparasmani sahu9630483273,9630483273
3,EP0203J2,2023-03-29,E-Shram Card_MP,SH0009HI,Benefit Received,2023-10-21,2023-10-21,Not Suspected,Madhya Pradesh,Damoh,PN026PZM,Rajesh Ahirwar,Male,6264372195,NaN,35,PG01Z1E0,Ahirwar,parash.sahu,Parasmani Sahu,Parasmani Sahu,E-Gov,e-shram card_mprajesh ahirwar6264372195,6264372195
4,EP021FB4,2023-04-03,Registration of farmers for training on natura...,SH000A1G,Open,NaT,NaT,Not Suspected,Madhya Pradesh,Damoh,PN0281HS,Lakhan Lal Pandey,Male,8889139573,NaN,51,PG020A2X,Pandey,parash.sahu,Parasmani Sahu,Parasmani Sahu,E-Gov,registration of farmers for training on natura...,8889139573


In [83]:
data4.drop("Screening Mobile_x", axis = 1, inplace=True)
data4.rename(columns={'Screening Mobile_y':'Screening Mobile'}, inplace=True)

In [84]:
data4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425300 entries, 0 to 425299
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Case Id                425300 non-null  object        
 1   Createdon              425300 non-null  datetime64[ns]
 2   Scheme/Doc             425300 non-null  object        
 3   Scheme/Doc GUID        425300 non-null  object        
 4   Status                 425300 non-null  object        
 5   Docket Submitted Date  396531 non-null  datetime64[ns]
 6   Benefit received Date  388401 non-null  datetime64[ns]
 7   HD Suspected Cases     425300 non-null  object        
 8   State                  425300 non-null  object        
 9   District               425300 non-null  object        
 10  Citizen GUID           425300 non-null  object        
 11  Citizen Name           425300 non-null  object        
 12  Gender                 425300 non-null  obje

In [85]:
data4.to_excel(r'C:\Users\akash\Documents\Haqdarshak\Work\Nassscom\1.0\NF 1.0 Data Till 18th Feb_pythonmapped.xlsx', index=False)